### Configuración Inicial

Para comenzar con la utilización de Spark y RDDs, es necesario realizar una configuración inicial:

1. **`findspark.init()`:** Este comando localiza e inicializa una instalación de Spark en tu máquina, permitiendo que Spark se integre con Jupyter.
2. **`SparkSession.builder...`:** Aquí estamos creando una instancia de SparkSession, que es el punto de entrada para cualquier funcionalidad en Spark. `master("local[*]")` le dice a Spark que corra localmente en tu máquina utilizando todos los núcleos disponibles. `appName("RDDPractice")` simplemente le da un nombre a tu SparkSession para identificación.


### Spark Core y RDD (Resilient Distributed Dataset)

In [1]:
# Importar findspark y inicializarlo
import findspark
findspark.init()

# Importar SparkSession y crear una instancia
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("RDDPractice").getOrCreate()


23/09/21 16:28:19 WARN Utils: Your hostname, MacBook-Air-de-Ivan.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
23/09/21 16:28:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 16:28:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/21 16:28:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/21 16:28:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/21 16:28:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### Creación de RDD

En esta sección, creamos un RDD a partir de una lista de nombres. Aquí están los pasos clave:

1. **`spark.sparkContext.parallelize(data)`:** Usamos el método `parallelize` para convertir una lista en un RDD. Este método distribuye los elementos de la lista a través de las diferentes particiones para procesamiento paralelo.
2. **`rdd_from_list.collect()`:** El método `collect` se utiliza para recuperar todos los elementos del RDD y mostrarlos. Es importante usar `collect` con precaución, ya que recopila todos los datos en la memoria del driver, lo que puede causar problemas si el RDD es muy grande.


In [2]:
# Creando un RDD desde una lista
data = ["Alice", "Bob", "Charlie", "David", "Eva"]
rdd_from_list = spark.sparkContext.parallelize(data)

# Mostrando el contenido del RDD
rdd_from_list.collect()

['Alice', 'Bob', 'Charlie', 'David', 'Eva']

In [3]:
spark

### Transformaciones en RDD

Las transformaciones permiten modificar los datos en el RDD sin alterar el RDD original. En este segmento, trabajamos con dos transformaciones comunes: `map` y `filter`.

1. **`map`:** Esta transformación aplica una función a cada elemento del RDD. En nuestro caso, hemos convertido cada nombre a mayúsculas utilizando una función lambda.
2. **`filter`:** Esta transformación devuelve un nuevo RDD formado por los elementos para los cuales la función proporcionada devuelve `True`. Aquí, hemos filtrado solo aquellos nombres que tienen más de 4 caracteres.
   
El resultado final muestra los nombres en mayúsculas y los nombres filtrados con más de 4 caracteres.


In [4]:
# Transformación: map
rdd_uppercase = rdd_from_list.map(lambda name: name.upper())

# Transformación: filter
rdd_filtered = rdd_uppercase.filter(lambda name: len(name) > 4)

# Mostrando resultados
rdd_uppercase.collect(), rdd_filtered.collect()


(['ALICE', 'BOB', 'CHARLIE', 'DAVID', 'EVA'], ['ALICE', 'CHARLIE', 'DAVID'])

### Acciones en RDD

Las acciones son operaciones que devuelven un valor al programa driver o escriben datos en un sistema de almacenamiento externo. En esta sección, ejecutamos dos acciones comunes: `count` y `first`.

1. **`count`:** Esta acción devuelve el número de elementos en el RDD. En nuestro ejemplo, contamos el número de nombres con más de 4 caracteres.
2. **`first`:** Esta acción devuelve el primer elemento del RDD. Aquí, recuperamos el primer nombre del RDD filtrado.

Las acciones son las que realmente desencadenan la ejecución en Spark. Hasta que se llama a una acción, las transformaciones se evalúan de manera "perezosa", lo que significa que no se ejecutan hasta que se necesita el resultado.


In [5]:
# Acciones: count y first
num_elements = rdd_filtered.count()
first_element = rdd_filtered.first()

num_elements, first_element


(3, 'ALICE')

### Transformación `flatMap`

La transformación `flatMap` es similar a `map`, pero cada entrada puede ser mapeada a cero o más salidas. En otras palabras, mientras que `map` produce un nuevo RDD con el mismo número de elementos que el original, `flatMap` puede producir un RDD con un número diferente de elementos.

En el ejemplo:
1. Creamos un RDD a partir de una lista de oraciones.
2. Usamos `flatMap` para dividir cada oración en palabras, lo que resulta en un RDD donde cada elemento es una palabra individual.
3. La acción `collect` se utiliza para visualizar el resultado.

Con `flatMap`, hemos "aplanado" las oraciones en un conjunto de palabras.


In [6]:
# Transformación: flatMap
sentences = ["Hello world", "I am learning Spark", "This is fun"]
rdd_sentences = spark.sparkContext.parallelize(sentences)
rdd_words = rdd_sentences.flatMap(lambda sentence: sentence.split())

# Acción: collect
rdd_words.collect()


['Hello', 'world', 'I', 'am', 'learning', 'Spark', 'This', 'is', 'fun']

### Acciones `reduce` y `take`

1. **`reduce`:** Esta acción toma una función que acepta dos argumentos y devuelve un solo valor, y utiliza esta función para reducir los elementos del RDD a un único valor. En nuestro caso, usamos `reduce` para sumar las longitudes de todas las palabras en el RDD.
2. **`take`:** Esta acción devuelve una lista con los primeros `n` elementos del RDD. En el ejemplo, recuperamos las tres primeras palabras del RDD.

Estas acciones son útiles para obtener información resumida o subconjuntos de datos del RDD para su análisis.


In [7]:
# Acciones: reduce y take
sum_of_lengths = rdd_words.map(lambda word: len(word)).reduce(lambda a, b: a + b)
first_three_words = rdd_words.take(3)

sum_of_lengths, first_three_words


(35, ['Hello', 'world', 'I'])

### Contar la Frecuencia de Palabras

El contar palabras es uno de los ejemplos canónicos en el procesamiento de datos distribuidos. En este ejemplo, contamos la frecuencia de palabras en un conjunto de textos utilizando RDDs.

1. **`flatMap`:** Primero, dividimos cada frase en palabras individuales y las convertimos a minúsculas para asegurarnos de que "Spark" y "spark" sean tratados como la misma palabra.
2. **`map`:** Luego, transformamos el RDD de palabras en un RDD de pares, donde cada palabra está asociada al número 1. Estos pares nos ayudarán a contar las ocurrencias de cada palabra.
3. **`reduceByKey`:** Esta transformación agrupa el RDD por palabras y suma sus valores para obtener la frecuencia de cada palabra.

El resultado final es un RDD de pares (palabra, frecuencia) que muestra cuántas veces aparece cada palabra en el conjunto de textos.


In [8]:
# Datos de entrada: Un RDD con varias frases
text_data = ["Spark is awesome", 
             "Spark is fast", 
             "I love Spark", 
             "Why is Spark popular?", 
             "Spark versus Hadoop"]

rdd_text = spark.sparkContext.parallelize(text_data)

# Dividimos el texto en palabras y lo ponemos en minúsculas
words = rdd_text.flatMap(lambda sentence: sentence.lower().split())

# Creamos un par RDD con cada palabra y el número 1
word_pairs = words.map(lambda word: (word, 1))

# Reducimos por clave para contar las ocurrencias de cada palabra
word_count = word_pairs.reduceByKey(lambda a, b: a + b)

# Resultado
word_count.collect()


[('i', 1),
 ('why', 1),
 ('hadoop', 1),
 ('spark', 5),
 ('popular?', 1),
 ('versus', 1),
 ('fast', 1),
 ('is', 3),
 ('awesome', 1),
 ('love', 1)]

### Operaciones de Conjunto en RDD

Operaciones de Set

Dado que un RDD es una colección de elementos, Spark admite operaciones de conjuntos tradicionales en RDDs. Estas operaciones son especialmente útiles cuando trabajamos con datos en los que queremos realizar uniones, intersecciones o diferencias.



1. **`union`:** Combina los elementos de dos RDDs. Si hay elementos duplicados, estos se mantienen.
2. **`intersection`:** Devuelve solo los elementos que están presentes en ambos RDDs.

En nuestro ejemplo, hemos creado dos RDDs con nombres de frutas. A través de `union`, obtenemos un RDD con todas las frutas de ambos RDDs. Con `intersection`, obtenemos un RDD con solo las frutas que aparecen en ambos RDDs originales.


In [9]:
rdd1 = spark.sparkContext.parallelize(["apple", "banana", "cherry"])
rdd2 = spark.sparkContext.parallelize(["banana", "cherry", "date", "fig"])

# Unión
union_rdd = rdd1.union(rdd2)

# Intersección
intersection_rdd = rdd1.intersection(rdd2)

union_rdd.collect(), intersection_rdd.collect()


(['apple', 'banana', 'cherry', 'banana', 'cherry', 'date', 'fig'],
 ['banana', 'cherry'])

### Particionamiento en RDD

El particionamiento es un proceso clave en Spark que determina cómo se distribuyen los datos en los clústeres. Un particionamiento adecuado puede mejorar significativamente la eficiencia de las operaciones en Spark.

El particionamiento controla cómo se distribuyen los datos físicamente a través de los nodos del clúster al almacenar un RDD o DataFrame.

1. **`getNumPartitions`:** Nos dice cuántas particiones tiene un RDD.
2. **`repartition`:** Nos permite cambiar el número de particiones. Es útil cuando se sabe que se va a trabajar con un gran volumen de datos y se quiere optimizar la distribución de esos datos.

En el ejemplo, primero verificamos el número de particiones del RDD original y luego reparticionamos el RDD para que tenga 2 particiones.


In [10]:
# Número de particiones del RDD original
num_partitions = rdd_text.getNumPartitions()

# Reparticionando el RDD a 2 particiones
repartitioned_rdd = rdd_text.repartition(2)

# Número de particiones después de reparticionar
new_num_partitions = repartitioned_rdd.getNumPartitions()

num_partitions, new_num_partitions


(8, 2)

### Persistencia en RDDs

Uno de los principales atractivos de Spark es su capacidad para almacenar datos en memoria entre operaciones, lo que permite realizar cálculos rápidos. Esta característica se denomina "persistencia" en Spark.

Cuando trabajamos con conjuntos de datos en los que realizamos múltiples operaciones (como filtrar, mapear, etc.), puede ser útil guardar el RDD en memoria, especialmente si vamos a reutilizar esos datos. Esto evita tener que recomputar todo desde el principio cada vez.

Niveles de Persistencia

Spark ofrece varios niveles de persistencia:

MEMORY_ONLY: Almacena el RDD solo en memoria.

MEMORY_AND_DISK: Almacena el RDD en memoria, pero si no cabe, guarda las particiones que no caben en el disco.

MEMORY_ONLY_SER: Similar a MEMORY_ONLY, pero con los datos serializados, lo que puede ser más eficiente en términos de espacio pero más lento en términos de acceso.

MEMORY_AND_DISK_SER: Combina las características de MEMORY_AND_DISK y MEMORY_ONLY_SER.

DISK_ONLY: Almacena el RDD solo en disco.

Es importante elegir el nivel de persistencia adecuado según las necesidades específicas del análisis y los recursos disponibles.


La persistencia permite almacenar un RDD en memoria o en disco, lo que facilita su reutilización en operaciones posteriores sin tener que recomputarlo.

1. **`persist()`:** Este método permite almacenar un RDD en memoria o en disco. Hay varios niveles de almacenamiento, siendo `StorageLevel.MEMORY_ONLY` uno de los más comunes, que guarda el RDD solo en memoria.
2. **`is_cached`:** Es una propiedad que verifica si un RDD está actualmente almacenado en memoria o no.

En el ejemplo, persistimos el RDD `rdd_text` en memoria y luego realizamos algunas operaciones sobre él. Finalmente, comprobamos si el RDD está realmente almacenado en memoria.


In [11]:
from pyspark import StorageLevel

# Persistir el RDD en memoria
rdd_text.persist(StorageLevel.MEMORY_ONLY)

# Realizamos algunas operaciones
count_words = rdd_text.flatMap(lambda sentence: sentence.split()).count()

# Verificar si el RDD está persistido
is_persisted = rdd_text.is_cached

count_words, is_persisted


(16, True)

In [12]:
spark.stop()